# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
!gdown --id '1-1ZbJqJqdmbswltY0AuZpvdhcNEGvGae' --output data.zip
!unzip data.zip
!ls 

Downloading...
From: https://drive.google.com/uc?id=1-1ZbJqJqdmbswltY0AuZpvdhcNEGvGae
To: /content/data.zip
372MB [00:03, 101MB/s] 
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [2]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [3]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [4]:
VAL_RATIO = 0.1

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
# train_x = train_x.reshape((train_x.shape[0],11,39))
# val_x = val_x.reshape((val_x.shape[0],11,39))

print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [5]:
# BATCH_SIZE = 128
BATCH_SIZE = 64


from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [6]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

153

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [14]:
import torch
import torch.nn as nn



class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        layer_input_size = 429 


        input_size = 39
        hidden_size = 256
        num_layers = 2
        num_classes = 39
        # bidirectional = True
        # self.num_directions = 2 if bidirectional else 1
        # layer_input_size = hidden_size * self.num_directions * 11 

        # self.num_layers = num_layers
        # self.hidden_size = hidden_size
        # self.gru = nn.GRU(input_size, hidden_size, num_layers, bidirectional=bidirectional, dropout=0.2, batch_first=True)
        # self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=bidirectional, dropout=0.2, batch_first=True)
        # self.bn_fn0  = nn.BatchNorm1d(layer_input_size)
        # self.layer0 = nn.Linear(layer_input_size, 128)
        # self.layer00 = nn.Linear(128, num_classes)


        self.layer1 = nn.Linear(layer_input_size, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 128)
        self.layer4 = nn.Linear(128, 256)
        self.layer5 = nn.Linear(256, 64)
        self.layer6 = nn.Linear(64, num_classes)

        self.act_fn = nn.RReLU() 
        self.act_fn2 = nn.Sigmoid()

        self.bn_fn1  = nn.BatchNorm1d(1024)
        self.bn_fn2  = nn.BatchNorm1d(512)
        self.bn_fn3  = nn.BatchNorm1d(128)
        self.bn_fn4  = nn.BatchNorm1d(256)
        self.bn_fn5  = nn.BatchNorm1d(64)

        # self.dropout = nn.Dropout(0.4)
        self.dropout = nn.Dropout(0.2)


    def forward(self, x):

        # h_0 = torch.zeros(self.num_layers*self.num_directions, x.size(0), self.hidden_size).to(device) 
        # c_0 = torch.zeros(self.num_layers*self.num_directions, x.size(0), self.hidden_size).to(device) 
        # hx = (h_0, c_0)
        # x, hx = self.lstm(x,hx)

        # hx = h_0
        # x, hx = self.gru(x,hx)
        # x = x[:,-1,:]
        # x = x[:,-2:,:] # batch_size,len
        # x = x.reshape(x.size(0),-1)
        # x = self.bn_fn0(x)
        # x = self.layer0(x)
        # x = self.act_fn(x)
        # x = self.dropout0(x) 
        # x = self.layer00(x) 
        # x = self.act_fn(x)

        
        x = self.layer1(x)
        x = self.bn_fn1(x)
        x = self.act_fn(x)
        x = self.dropout(x) 

        x = self.layer2(x)
        # x = self.bn_fn2(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer3(x)
        x = self.bn_fn3(x)
        x = self.act_fn(x)
        x = self.dropout(x) 

        x = self.layer4(x)
        # x = self.bn_fn4(x)
        x = self.act_fn(x)
        x = self.dropout(x) 

        x = self.layer5(x)
        # x = self.bn_fn5(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer6(x)
        x = self.act_fn(x)

        return x

## Training

In [15]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [16]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [19]:
# fix random seed for reproducibility
same_seeds(5)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 1000               # number of training epoch
learning_rate = 0.001       # learning rate
# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.5, weight_decay=0.00001 )


DEVICE: cuda


In [ ]:
# start training

best_acc = 0.0

scheduler1 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500)

# scheduler1 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.1,patience=10)

early_stop_cnt = 0

for epoch in range(num_epoch):

    
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step()

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))
            # scheduler1.step(val_loss) 
            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
                early_stop_cnt = 0
            else:
              early_stop_cnt += 1
            # if early_stop_cnt > 10:
            # Stop training if your model stops improving for "config['early_stop']" epochs.
              # break
            # threshold = 0.8
            # if train_acc/len(train_set) > train_loss/len(train_loader) + threshold :
            #   print('Train{:3.6f}>Val{:3.6f}'.format(train_acc/len(train_set), val_acc/len(val_set)))
            #   break
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
        # if train_acc/len(train_set)>train_loss/len(train_loader)+0.1:
        #   break

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/1000] Train Acc: 0.554488 Loss: 1.474710 | Val Acc: 0.655739 loss: 1.100121
saving model with acc 0.656
[002/1000] Train Acc: 0.611016 Loss: 1.267118 | Val Acc: 0.677098 loss: 1.019084
saving model with acc 0.677
[003/1000] Train Acc: 0.628136 Loss: 1.205187 | Val Acc: 0.691310 loss: 0.967216
saving model with acc 0.691
[004/1000] Train Acc: 0.638646 Loss: 1.169168 | Val Acc: 0.698424 loss: 0.939928
saving model with acc 0.698
[005/1000] Train Acc: 0.645061 Loss: 1.143405 | Val Acc: 0.702238 loss: 0.924723
saving model with acc 0.702
[006/1000] Train Acc: 0.650973 Loss: 1.123965 | Val Acc: 0.707457 loss: 0.906181
saving model with acc 0.707
[007/1000] Train Acc: 0.654649 Loss: 1.109060 | Val Acc: 0.709173 loss: 0.898092
saving model with acc 0.709
[008/1000] Train Acc: 0.658327 Loss: 1.097790 | Val Acc: 0.711888 loss: 0.888607
saving model with acc 0.712
[009/1000] Train Acc: 0.660560 Loss: 1.088302 | Val Acc: 0.714051 loss: 0.883875
saving model with acc 0.714
[010/1000] Train Ac

*斜體文字*## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
# test = test.reshape((test.shape[0],11,39))
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

Reference
This code is completely written by NTUEE.
Copying or reusing this code is required to specify the original author.

E.g.
Source: NTUEE (https://colab.research.google.com/github/ga642381/ML2021-Spring/blob/main/HW02/HW02-1.ipynb)